# Model Training
We are going to train a sentiment analysis Model to determine if a Yelp review is either Positive Or Negative.

This model training session is also going to be used as a case study to see how accurate the XGBoost Framework can be used in this type of scenario. This is typically not an NLP framework used.

This session will teach the importance of Preprocessing data as well as how to train a model. We will use the popular nltk project for preprocessing.

In [ ]:
# Install additional libraries
!pip install nltk

In [ ]:
### RESTART KERNEL FoR NEW LIBRARY TO TAKE ####

# Preparing our Data
We just created a dataset and saved it as 'training_data.csv' in Session 2. We are going to use this to train our model. We will need to clean up and preprocess our data. We will use nltk for this.

In [1]:
# Import our dataset to Pandas DF
import pandas as pd

data = pd.read_csv('training_data.csv', index_col=0)
data

,stars,text,sentiment
0,5,My wife took me here on my birthday for breakf...,positive
1,5,I have no idea why some people give bad review...,positive
2,4,love the gyro plate. Rice is so good and I als...,positive
3,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",positive
4,5,General Manager Scott Petello is a good egg!!!...,positive
...,...,...,...
9995,3,First visit...Had lunch here today - used my G...,positive
9996,4,Should be called house of deliciousness!\n\nI ...,positive
9997,4,I recently visited Olive and Ivy for business ...,positive
9998,2,My nephew just moved to Scottsdale recently so...,negative


# Preprocess with nltk
We will use stopwords and WordNetLemmatizer.

**Stop words** - Stop words are a set of commonly used words in a language. Examples of stop words in English are “a,” “the,” “is,” “are,” etc. Stop words are commonly used in Text Mining and Natural Language Processing (NLP) to eliminate words that are so widely used that they carry very little useful information.

**Word Lemmatizer** - Reduce a word to its root form, also called a lemma. For example, the verb "running" would be identified as "run." Lemmatization studies the morphological, or structural, and contextual analysis of words.


In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Download corpora (shit ton of text)
nltk.download('stopwords')
nltk.download('wordnet')

# English stop words here
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/weston/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/weston/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# Function that cleans up the text. Remove things like punctuation, convert to lowercase, lemmatize and remove stop words

def clean_text(text):
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text, re.UNICODE)
    # convert to lowercase
    text = text.lower()
    # Lemmatize
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")] 
    # remove stop words
    text = [word for word in text if not word in stop_words] 
    # Bring the list back into a string
    text = " ".join(text)
    
    return text

In [4]:
# Apply text cleaning function above to each row (lambda) of our dataset 
# Here we add an additional row just to demonstrate.
# This is the column we will train our model on!
data['cleaned_text'] = data.text.apply(lambda x: clean_text(x))

In [5]:
# Read it an notice the differences between text and cleaned_text columns
data

,stars,text,sentiment,cleaned_text
0,5,My wife took me here on my birthday for breakf...,positive,wife took birthday breakfast wa excellent wea...
1,5,I have no idea why some people give bad review...,positive,idea people give bad review place go show plea...
2,4,love the gyro plate. Rice is so good and I als...,positive,love gyro plate rice good also dig candy selec...
3,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",positive,rosie dakota love chaparral dog park convenien...
4,5,General Manager Scott Petello is a good egg!!!...,positive,general manager scott petello good egg go deta...
...,...,...,...,...
9995,3,First visit...Had lunch here today - used my G...,positive,first visithad lunch today used groupon \n\n...
9996,4,Should be called house of deliciousness!\n\nI ...,positive,called house deliciousness\n\ni could go item ...
9997,4,I recently visited Olive and Ivy for business ...,positive,recently visited olive ivy business last week ...
9998,2,My nephew just moved to Scottsdale recently so...,negative,nephew moved scottsdale recently bunch friend ...


# Feature Extraction
In natural language processing (NLP), feature extraction is a fundamental task that involves converting raw text data into a format that can be easily processed by machine learning algorithms.

Machines need read numerical data!

In [6]:
# Use Sklearn feature extractor
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

# Vectorize (convert to numbers) on the cleaned_text column and make it an array to pass to our model for training
vectorizer = TfidfVectorizer(max_features=5000)
tfid_obj = vectorizer.fit_transform(data['cleaned_text'])

# Save our fitted vectorizer using pickle
vec_file = 'vectorizer.pickle'
pickle.dump(vectorizer, open(vec_file, 'wb'))


In [7]:
# Convert the features to an array and Read the array just to see
data_features = tfid_obj.toarray()
data_features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
# We also need to make our sentiments a numerical value. To keep this simple we will make positive a 1 and negative a 0.
def encode_sentiment(sentiment):
    if sentiment == 'positive':
        sentiment_value = 1
    else:
        sentiment_value = 0

    return int(sentiment_value)



In [9]:
# Add new column with encodings 
data['encoded_sentiment'] = data.sentiment.apply(lambda x: encode_sentiment(x))

In [10]:
# Take a look at dataset
data

,stars,text,sentiment,cleaned_text,encoded_sentiment
0,5,My wife took me here on my birthday for breakf...,positive,wife took birthday breakfast wa excellent wea...,1
1,5,I have no idea why some people give bad review...,positive,idea people give bad review place go show plea...,1
2,4,love the gyro plate. Rice is so good and I als...,positive,love gyro plate rice good also dig candy selec...,1
3,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",positive,rosie dakota love chaparral dog park convenien...,1
4,5,General Manager Scott Petello is a good egg!!!...,positive,general manager scott petello good egg go deta...,1
...,...,...,...,...,...
9995,3,First visit...Had lunch here today - used my G...,positive,first visithad lunch today used groupon \n\n...,1
9996,4,Should be called house of deliciousness!\n\nI ...,positive,called house deliciousness\n\ni could go item ...,1
9997,4,I recently visited Olive and Ivy for business ...,positive,recently visited olive ivy business last week ...,1
9998,2,My nephew just moved to Scottsdale recently so...,negative,nephew moved scottsdale recently bunch friend ...,0


# Split the data
The train-validation-test split is a strategy that divides a dataset into three essential subsets: the training set, the validation set, and the test set. Each subset serves a distinct purpose.

In [11]:
from sklearn.model_selection import train_test_split

# X_train, X_test will be our array above
# y_train, y_test is our encoded_sentiment. y is the result (prediction_ you are going for)
X_train, X_test, y_train, y_test = train_test_split(data_features, data['encoded_sentiment'], test_size=0.2, random_state=42)

In [12]:
X_train, y_train

(array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.50226399, 0.        , ..., 0.        , 0.        ,
         0.        ]]),
 9254    1
 1561    1
 1670    1
 6087    1
 6669    1
        ..
 5734    1
 5191    1
 5390    1
 860     1
 7270    1
 Name: encoded_sentiment, Length: 8000, dtype: int64)

# Train the Model
Now the fun part!!!! Train an model!!!

Now we are going to train an XGBoost model our our training data (X_train, y_train)

In [13]:
import xgboost as xgb

# Use the Classifier. We can adjust the parameters here to try to get better results.
model = xgb.XGBClassifier(max_depth=10, n_estimators=1000, learning_rate=0.01)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [14]:
# Save our model locally
model.save_model("model.json")

In [15]:
# Now we can do model evaluations using our Test Dataset
# Use the model to run predictions across 2000K rows
predictions = model.predict(X_test)

In [16]:
# This will output the encoded sentiments
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
# There are several ways we can see the results with different 

In [17]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.82      0.36      0.50       308
           1       0.89      0.99      0.94      1692

    accuracy                           0.89      2000
   macro avg       0.86      0.67      0.72      2000
weighted avg       0.88      0.89      0.87      2000



# Inference (Real Tests)
Now lets run through the process of using some real text to get predictions. We must follow the same steps we did for training as we do to get results.

**Inference** - Applying a machine learning model to a dataset and generating an output or “prediction”. This output might be a numerical score, a string of text, an image, or any other structured or unstructured data.

In [ ]:
# Load the model if needed
# import xgboost as xgb

# model = xgb.XGBClassifier(max_depth=10, n_estimators=1000, learning_rate=0.01)
# model.load_model("model.json")

In [18]:
# Follow same steps as dataprocessing for model
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Download corpora (shit ton of text)
nltk.download('stopwords')
nltk.download('wordnet')

# English stop words here
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/weston/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/weston/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
# Load our vectorizer
loaded_vectorizer = pickle.load(open('vectorizer.pickle', 'rb'))

def clean_and_vectorize(text):
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text, re.UNICODE)
    # convert to lowercase
    text = text.lower()
    # Lemmatize
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")] 
    # remove stop words
    text = [word for word in text if not word in stop_words] 
    # Bring the list back into a string
    text = " ".join(text)

    # Vectorize from our vectorizer created above
    data_features = loaded_vectorizer.transform([text])
    # Create an array as it expects
    data_features = data_features.toarray()

    # Get the prediciton 
    prediction = model.predict(data_features)[0]

    # 1 is positive 0 is negative
    if prediction == 1:
        sentiment = 'positive'
    else: 
        sentiment = 'negative'

    return sentiment

In [20]:
text = 'This place is the greatest on earth!'
prediction = clean_and_vectorize(text)
prediction

'positive'

In [21]:
text = """
Alqueria surpassed my expectations ten fold. You can tell that their food is authentic farm-to-table and is just incredibly fresh.
We ordered the shrimp and drunken goat cheese as our appetizers and they proved we made the right decision in choosing Alqueria for dinner. The shrimp, with the oil that it is in, is unrivaled. I would come back just to eat more of this! The lamb and spaghetti squash were also very good. The lamb fell right off the bone.
Our server was very sweet to us, offered her suggestions, and always checked in.
I do love the intimate feel of the restaurant, however, reservations are necessary since it is a smaller place.
The menu changes often which I think is a fun concept, but I am really hoping they don't ever take the shrimp off the menu!
Braised Lamb Shank & Local Spaghetti Squash
"""

prediction = clean_and_vectorize(text)
prediction

'positive'

In [22]:
text = """
Wow! Downright awful! Some background: I have avoided eating Panera basically my entire life because it's terrible. However! In Orlando a group of friends wanted to go to Panera, so I went. It was awesome! So good that I told several people about it and was like we have to go when we're all back in Columbus because it was so good. Which was so surprising. Well. Here we are. And it was awful. Terrible. Put together terribly and tasted like feet and rot. Stay away. Better to not eat than to waste money on this garbage.
"""
prediction = clean_and_vectorize(text)
prediction

'negative'